In [6]:
#import dependencies
import pandas as pd
from scipy.interpolate import interp1d
import numpy as np
from datetime import timedelta

#Library to plot data
import plotly.graph_objects as go

#import data
MAPE_data = pd.read_csv('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/MAPE_all_Models.csv')
ISalpha = pd.read_csv('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/ISalpha_all_Models.csv')

data_points_MAPE = 109
t_data_points_MAPE = np.linspace(1,data_points_MAPE,data_points_MAPE)

#Interpolating weekly MAPE SIR data to daily
temp_MAPE_SIR_weekly = MAPE_data['MAPE_SIR_weekly']
temp_MAPE_SIR_weekly.truncate(0,14,copy = False)

SIR_MAPE_week2daily = []
for week in range(len(temp_MAPE_SIR_weekly)-1):
    SIR_MAPE_week2daily.extend(np.linspace(temp_MAPE_SIR_weekly[week],temp_MAPE_SIR_weekly[week+1],8)[:7])
    
#Interpolating weekly ISalpha SIR data to daily
temp_ISalpha_SIR_weekly = ISalpha['ISalpha_SIR_weekly']
temp_ISalpha_SIR_weekly.truncate(0,14,copy = False)

SIR_ISalpha_week2daily = []
for week in range(len(temp_ISalpha_SIR_weekly)-1):
    SIR_ISalpha_week2daily.extend(np.linspace(temp_ISalpha_SIR_weekly[week],temp_ISalpha_SIR_weekly[week+1],8)[:7])
    

In [7]:
MAPEfig1 = go.Figure()

#SIR MAPE week to daily
MAPEfig1.add_trace(go.Scatter(x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=data_points_MAPE)), 
                    y=SIR_MAPE_week2daily,
                    mode='lines',
                    name='SIR veckolig modell (interpolerad)'))

#SIR MAPE daily
MAPEfig1.add_trace(go.Scatter(x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=data_points_MAPE)), 
                    y=MAPE_data['MAPE_SIR_daily'],
                    mode='lines',
                    name='SIR daglig modell'))



MAPEfig1.update_layout(
                    title='MAPE mot ökande inskrivningsdata',
                    xaxis_title='Date',
                    yaxis_title='MAPE %',
                    autosize=False,
                    width=1000,
                    xaxis_range=["2020-03-01",pd.to_datetime('2020-03-01')+timedelta(days=data_points_MAPE)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
    )
) 
MAPEfig1.show()

MAPEfig2 = go.Figure()


#logistiska MAPE
MAPEfig2.add_trace(go.Scatter(x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=data_points_MAPE)), 
                    y=MAPE_data['MAPE_logistiska'],
                    mode='lines',
                    name='Logistisk modell'))

#logistiska mobilitet MAPE
MAPEfig2.add_trace(go.Scatter(x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=data_points_MAPE)), 
                    y=MAPE_data['MAPE_logistiska_mobilitet'],
                    mode='lines',
                    name='Logistisk modell med mobilitet'))

#logistiska tak MAPE
MAPEfig2.add_trace(go.Scatter(x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=data_points_MAPE)), 
                    y=MAPE_data['MAPE_logistiska_tak'],
                    mode='lines',
                    name='Logistisk modell med tak'))

#logistiska mobilitet tak MAPE
MAPEfig2.add_trace(go.Scatter(x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=data_points_MAPE)),
                    y=MAPE_data['MAPE_logistiska_mobilitet_tak'],
                    mode='lines',
                    name='Logistisk modell med mobilitet och tak'))

MAPEfig2.update_layout(
                    title='MAPE mot ökande inskrivningsdata',
                    xaxis_title='Date',
                    yaxis_title='MAPE %',
                    autosize=False,
                    width=1000,
                    xaxis_range=["2020-03-01",pd.to_datetime('2020-03-01')+timedelta(days=data_points_MAPE)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
    )
) 
MAPEfig2.show()

In [12]:
ISalphafig1 = go.Figure()

#SIR MAPE week to daily
ISalphafig1.add_trace(go.Scatter(x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=data_points_MAPE)), 
                    y=SIR_ISalpha_week2daily,
                    mode='lines',
                    name='SIR veckolig modell (interpolerad)'))

#SIR MAPE daily
ISalphafig1.add_trace(go.Scatter(x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=data_points_MAPE)), 
                    y=ISalpha['ISalpha_SIR_daily'],
                    mode='lines',
                    name='SIR daglig modell'))



ISalphafig1.update_layout(
                    title='ISalpha mot ökande inskrivningsdata',
                    xaxis_title='Date',
                    yaxis_title='ISalpha score',
                    autosize=False,
                    width=1000,
                    xaxis_range=["2020-03-01",pd.to_datetime('2020-03-01')+timedelta(days=data_points_MAPE)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
    )
) 
ISalphafig1.show()

ISalphafig2 = go.Figure()


#logistiska MAPE
ISalphafig2.add_trace(go.Scatter(x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=data_points_MAPE)), 
                    y=ISalpha['ISalpha_logistiska'],
                    mode='lines',
                    name='Logistisk modell'))

#logistiska mobilitet MAPE
ISalphafig2.add_trace(go.Scatter(x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=data_points_MAPE)), 
                    y=ISalpha['ISalpha_logistiska_mobilitet'],
                    mode='lines',
                    name='Logistisk modell med mobilitet'))

#logistiska tak MAPE
ISalphafig2.add_trace(go.Scatter(x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=data_points_MAPE)), 
                    y=ISalpha['ISalpha_logistiska_tak'],
                    mode='lines',
                    name='Logistisk modell med tak'))

#logistiska mobilitet tak MAPE
ISalphafig2.add_trace(go.Scatter(x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=data_points_MAPE)),
                    y=ISalpha['ISalpha_logistiska_mobilitet_tak'],
                    mode='lines',
                    name='Logistisk modell med mobilitet och tak'))

ISalphafig2.update_layout(
                    title='ISalpha mot ökande inskrivningsdata',
                    xaxis_title='Date',
                    yaxis_title='ISalpha score',
                    autosize=False,
                    width=1000,
                    xaxis_range=["2020-03-01",pd.to_datetime('2020-03-01')+timedelta(days=data_points_MAPE)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
    )
) 
ISalphafig2.show()